In [1]:
class Node:
    def __init__(self, data, depth=0, lchild=None, rchild=None):
        self.data = data
        self.depth = depth
        self.lchild = lchild
        self.rchild = rchild
class KdTree:
    def __init__(self):
        self.KdTree = None
        self.n = 0
        self.nearest = None

    def create(self, dataSet, depth=0):
        if len(dataSet) > 0:
            m, n = np.shape(dataSet)
            self.n = n - 1
            axis = depth % self.n
            mid = int(m / 2)
            dataSetcopy = sorted(dataSet, key=lambda x: x[axis])
            node = Node(dataSetcopy[mid], depth)
            if depth == 0:
                self.KdTree = node
            node.lchild = self.create(dataSetcopy[:mid], depth+1)
            node.rchild = self.create(dataSetcopy[mid+1:], depth+1)
            return node
        return None

    def preOrder(self, node):
        if node is not None:
            print(node.depth, node.data)
            self.preOrder(node.lchild)
            self.preOrder(node.rchild)

    def search(self, x, count=1):
        nearest = []
        for i in range(count):
            nearest.append([-1, None])
    
        self.nearest = np.array(nearest)

        def recurve(node):
            if node is not None:
                axis = node.depth % self.n
                daxis = x[axis] - node.data[axis]
                if daxis < 0:
                    recurve(node.lchild)
                else:
                    recurve(node.rchild)

                dist = sqrt(sum((p1 - p2) ** 2 for p1, p2 in zip(x, node.data)))
                for i, d in enumerate(self.nearest):
                    if d[0] < 0 or dist < d[0]:
                        self.nearest = np.insert(self.nearest, i, [dist, node], axis=0)
                        self.nearest = self.nearest[:-1]
                        break

                n = list(self.nearest[:, 0]).count(-1)
                if self.nearest[-n-1, 0] > abs(daxis):
                    if daxis < 0:
                        recurve(node.rchild)
                    else:
                        recurve(node.lchild)

        recurve(self.KdTree)

        knn = self.nearest[:, 1]
        return self.nearest

In [2]:
import numpy as np
from math import sqrt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mpl_toolkits.axisartist.axislines as axislines
import matplotlib as mpl
from sklearn import preprocessing
min_max_scaler=preprocessing.MinMaxScaler()
mpl.rcParams['font.sans-serif'] = [u'SimHei']
mpl.rcParams['axes.unicode_minus'] = False
from sklearn.model_selection import KFold
kf=KFold(n_splits=10)

def DDF(df):    
    data2=np.array(df.drop('label',axis=1))
    dmax=[]
    for m in range(0,len(data2)-1):
        for n in range(m+1,len(data2)):
            dmax.append(sqrt(sum((data2[m] - data2[n]) ** 2 )))
    a = np.percentile(dmax,10) #密度阈值(%10分位数)
    k=7
    x1 = np.array([[1,1],[1,-1]])
    x2 = np.array([1,np.floor(k / 3)/ k])
    (Ps_,Pd_)=np.linalg.solve(x1,x2)
    H_ = -Ps_* np.log2(Ps_)-Pd_*np.log2(Pd_) #标签混合程度阈值
    Fh,Re=[],[]
    data=np.array(df)
    for rate in [0.05,0.1,0.2,0.3,0.4]:
        fh_temp,re_temp=[],[]
        for train_index, test_index in kf.split(data):
            train=data[train_index]
            test=data[test_index]
            train=pd.DataFrame(train)
            train.columns=df.columns
            Dn= []
            noise_set=pd.DataFrame(columns=train.columns)
            label_set=list(set(train.label))
            for r in label_set:    
                noise_set=pd.concat([noise_set,train[train.label==r].sample(frac=rate,replace=False)]) #选取噪声比例 
            train_cut=train[~train.index.isin(noise_set.index)]
            no=[]
            for j in range(0,len(noise_set)):
                no.append(np.random.choice(label_set))
            noise_set['label2']=no
            noise_set.reset_index(drop=True,inplace=True)
            for l in range(0,len(noise_set)):  #随机替换标签
                label_set=list(set(train.label))
                if (noise_set.loc[l,'label']==noise_set.loc[l,'label2']):
                    label_set.remove(noise_set.loc[l,'label'])
                    noise_set.loc[l,'label2']=np.random.choice(label_set) #从剩余列表中随机选择
            noise=noise_set.drop(['label'],axis=1)
            noise.rename(columns={'label2':'label'}, inplace = True)
            train= pd.concat([train_cut,noise],axis=0,ignore_index=True) #含噪声数据集
            train_set=np.array(train)
            kdt = KdTree()
            kdt.create(train_set)  
            for x in train_set:
            #plt.scatter(x[0], x[1], c='red', marker='x')  # 测试点
                near= kdt.search(x[:-1], k+1)  # 设置临近点的个数
                density = 0
                t,hon,hen,dist_hon,dist_hen,DoD,DRL= 0,0,0,0,0,0,0
                for i in range(1,k+1):
                    if x[-1] == near[i][1].data[-1]:
                        t += 1
                        hon += 1
                        dist_hon += near[i][0]
                    else:
                        hen += 1
                        dist_hen += near[i][0]
                    density += near[i][0]  #密度   
                DoD = abs(dist_hen - dist_hon) #相异性差值(要加绝对值)
                DRL = (hen - hon) / k #标签异同差  
                Ps = t / k
                Pd = 1 - Ps
                if (Ps==1)|(Ps==0):
                    H = 0
                else:
                    H = -Ps* np.log2(Ps)-Pd*np.log2(Pd)
              #  print('densit:',density)
                if (density <= a): #高密度区域
                   # print('高密度区')
                    if DRL > 0:
                        Dn.append(list(x))
                       # print('高密度区噪声')
                else:
                    if (H < H_): #低密度单一标签区域
                        if DRL > 0:
                            Dn.append(list(x))
                           # print('低密度单一标签噪声'
                    else: #低密度混合标签区  
                      #  print('DoD',DoD)
                        DoD_ = density / k
                       # print('DoD阈值:',DoD_)
                        if DoD >= DoD_:
                            hon_sa,hen_sa,DRL_sa=0,0,0
                            near2=[]
                            for j in range(1,k+1):
                                if near[j][0] <= a: #该点到待测点距离小于a
                                    near2.append(near[j])
                            for n2 in near2:
                                if (x[-1]==n2[1].data[-1]):
                                    hon_sa +=1
                                else:
                                    hen_sa +=1
                            DRL_sa = hen_sa - hon_sa
                            if DRL_sa > 0:        
                                Dn.append(list(x))
                               # print('混合标签区噪声')
                            elif DRL_sa ==0 and len(near2)<k:
                                if x[-1] != near[len(near2)+1][1].data[-1]:
                                    Dn.append(list(x))  

            noise_list=noise.values.tolist()
            TP = 0
            for c in Dn:
                if c in noise_list:
                    TP += 1
            FP = len(Dn) - TP
            FN = len(noise_list) - TP
            precision= TP / (TP + FP)
            recall= TP / (TP + FN)
            fh= 2* (precision * recall) / (precision + recall) #F1值
            re = len(Dn) / len(train_set)#移除率
            fh_temp.append(fh)
            re_temp.append(re)
        Fh.append(np.mean(fh_temp))
        Re.append(np.mean(re_temp))
    return Fh,Re

In [3]:
def RDS(df):    
    k=5
    x1 = np.array([[1,1],[1,-1]])
    x2 = np.array([1,np.floor(k / 3)/ k])
    (Ps_,Pd_)=np.linalg.solve(x1,x2)
    H_ = -Ps_* np.log2(Ps_)-Pd_*np.log2(Pd_) #标签混合程度阈值
    data2=np.array(df.drop('label',axis=1))
    dmax=[]
    for m in range(0,len(data2)-1):
        for n in range(m+1,len(data2)):
            dmax.append(sqrt(sum((data2[m] - data2[n]) ** 2 )))
    a = np.percentile(dmax,10) #密度阈值(%10分位数)
    Fh,Re=[],[]
    data=np.array(df)
    for rate in [0.05,0.1,0.2,0.3,0.4]:
        fh_temp,re_temp=[],[]
        for train_index, test_index in kf.split(data):
            train=data[train_index]
            test=data[test_index]
            train=pd.DataFrame(train)
            train.columns=df.columns
            Dn,Dn_filted= [],[]
            noise_set=pd.DataFrame(columns=train.columns)
            label_set=list(set(train.label))
            for r in label_set:    
                noise_set=pd.concat([noise_set,train[train.label==r].sample(frac=rate, replace=False)]) #选取噪声比例 
            train_cut=train[~train.index.isin(noise_set.index)]
            no=[]
            for j in range(0,len(noise_set)):
                no.append(np.random.choice(label_set))
            noise_set['label2']=no
            noise_set.reset_index(drop=True,inplace=True)
            for l in range(0,len(noise_set)):  #随机替换标签
                label_set=list(set(train.label))
                if (noise_set.loc[l,'label']==noise_set.loc[l,'label2']):
                    label_set.remove(noise_set.loc[l,'label'])
                    noise_set.loc[l,'label2']=np.random.choice(label_set) #从剩余列表中随机选择
            noise=noise_set.drop(['label'],axis=1)
            noise.rename(columns={'label2':'label'}, inplace = True)
            train= pd.concat([train_cut,noise],axis=0,ignore_index=True) #含噪声数据集
            train_set=np.array(train)
            kdt = KdTree()
            kdt.create(train_set)  
            for x in train_set:
                near = kdt.search(x[:-1], k+1)  # 设置临近点的个数
                density = 0
                t,hon,hen,dist_hon,dist_hen,DoD,DRL= 0,0,0,0,0,0,0
                for i in range(1,k+1):
                    if x[-1] == near[i][1].data[-1]:
                        t += 1
                        hon += 1
                        dist_hon += near[i][0]
                    else:
                        hen += 1
                        dist_hen += near[i][0]
                    density += near[i][0]  #密度   

                DoD = abs(dist_hen - dist_hon) #相异性差值(要加绝对值)
                DRL = (hen - hon)  #标签异同差   
                Ps = t / k
                Pd = 1 - Ps
                if (Ps==1)|(Ps==0):
                    H = 0
                else:
                    H = -Ps* np.log2(Ps)-Pd*np.log2(Pd)
              #  print('densit:',density)
                if (density <= a): #高密度区域
                    if DRL > 0:
                        Dn.append(list(x))
                else:
                    if (H < H_): #低密度单一标签区域
                        if DRL > 0:
                            Dn.append(list(x))
                           # print('低密度单一标签噪声')
                    else: #低密度混合标签区  
                        DoD_ = density / k
                        if DoD >= DoD_:
                            train_hon,train_hen=[],[]
                            for j in train_set:
                                if j[-1]==x[-1]:
                                    train_hon.append(j)
                                else:
                                    train_hen.append(j)
                            kdt_hon=KdTree()
                            kdt_hen=KdTree()
                            kdt_hon.create(train_hon)
                            kdt_hen.create(train_hen)
                            near_hon=kdt_hon.search(x,k+1)
                            near_hen=kdt_hen.search(x,k)
                            density_hon,density_hen=0,0
                            for hon in near_hon:
                                density_hon += hon[0]
                            for hen in near_hen:
                                density_hen += hen[0]
                            if density_hon / density_hen > 1:
                                Dn.append(list(x))                 
            noise_neighbor=[]                   
            for e in Dn:  # 计算噪声分数
                e=np.array(e)
                near2=kdt.search(e[:-1],k+1) #k近邻
                for i in range(1,k+1):
                    noise_neighbor.append(list(near2[i][1].data))
            for e in Dn:
                e=np.array(e)
                near3=kdt.search(e[:-1],k+1)
                t_e=noise_neighbor.count(list(e)) #处于其它噪声近邻的次数
                confidence_e= 1 / np.sqrt(1+t_e*t_e)#e的置信度
                neighborhood_e=0
                for i in range(1,k+1):
                    t_ei=noise_neighbor.count(list(near3[i][1].data))
                    confidence_ei=1 / np.sqrt(1+t_ei*t_ei) 
                    if near3[i][1].data[-1]==e[-1]:
                        different_class=-1
                    else:
                        different_class=1
                    if list(near3[i][1].data) in Dn: 
                        isnoise=1
                    else:
                        isnoise=-1
                    near4=kdt.search(near3[i][1].data[:-1],k+1)
                    n_ei=0
                    for j in range(1,k+1):
                        if list(near4[j][1].data) in Dn:
                            n_ei += 1
                    clean_ei= (k + isnoise*(n_ei- k)) / (2*k) #纯净度
                    neighborhood_e += (clean_ei*confidence_ei*different_class)/k 
                NS_e=confidence_e * neighborhood_e #噪声得分
                if NS_e > 0:
                    Dn_filted.append(list(e))
            noise_list=noise.values.tolist()
            TP = 0
            for c in Dn_filted:
                if c in noise_list:
                    TP += 1
            FP = len(Dn_filted) - TP
            FN = len(noise_list) - TP
            precision= TP / (TP + FP)
            recall= TP / (TP + FN)
            fh= 2* (precision * recall) / (precision + recall) #F1值
            re = len(Dn_filted) / len(train_set)#移除率
            fh_temp.append(fh)
            re_temp.append(re)
        Fh.append(np.mean(fh_temp))
        Re.append(np.mean(re_temp))
    return Fh,Re

In [4]:
import mpl_toolkits.axisartist.axislines as axislines
import matplotlib as mpl
mpl.rcParams['font.sans-serif'] = [u'SimHei']
mpl.rcParams['axes.unicode_minus'] = False
%matplotlib

Using matplotlib backend: Qt5Agg


In [5]:
def plot_res(res_DDF,res_RDS,title):
    noise_rate=[5,10,20,30,40]
    fig = plt.figure(figsize=(5,8))
    ax1 = axislines.Subplot(fig, 2,1,1)
    fig.add_subplot(ax1)
    ax1.set_xticks(noise_rate)
    ax1.axis([0,40,0,1])
    plt.plot(noise_rate,res_DDF[0],'ro-',label='DDF')
    plt.plot(noise_rate,res_RDS[0],'b*-.',label='DDF-SCORE')
    plt.xlabel('noise rate/%')
    plt.ylabel('F1')
    plt.title(title)
    plt.legend(frameon=False)
    plt.show()
    ax2 = axislines.Subplot(fig, 2,1,2)
    fig.add_subplot(ax2)
    ax2.set_xticks([5,10,20,30,40])
    ax2.set_yticks([0,0.2,0.4,0.6,0.8])
    ax2.axis([0,40,0,0.8])
    plt.plot(noise_rate,res_DDF[1],'ro-',label='DDF')
    plt.plot(noise_rate,res_RDS[1],'b*-.',label='DDF-SCORE')
    plt.xlabel('noise rate/%')
    plt.ylabel('Re')
   # plt.title('不同噪声下的过滤值Re',fontsize=12)
    plt.legend(frameon=False)
    plt.show()
    plt.tight_layout(2) #设置子图间隔
    #plt.suptitle(title)
    plt.show()

#iris数据集

In [6]:
%%time
from sklearn.datasets import load_iris
iris = load_iris()
iris_sample= pd.DataFrame(iris.data, columns=iris.feature_names)
iris_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(iris_sample)))
iris_sample['label'] = iris.target
iris_DDF=DDF(iris_sample)
iris_RDS=RDS(iris_sample)
#plot_res(iris_DDF,iris_RDS,'(a)Iris ')
#plt.savefig(r'./对比实验结果图F1/iris.png')

Wall time: 40.9 s


In [7]:
iris_DDF

([0.759047619047619,
  0.8542977788749235,
  0.8796750592984169,
  0.8435148944910698,
  0.7953985764649758],
 [0.07333333333333333,
  0.1355555555555556,
  0.24148148148148146,
  0.3540740740740741,
  0.4925925925925926])

In [8]:
iris_RDS

([0.8004804998922646,
  0.8893125902647594,
  0.8822649041913078,
  0.8650143130155514,
  0.7861632180996904],
 [0.06814814814814815,
  0.1162962962962963,
  0.19925925925925925,
  0.28888888888888886,
  0.36444444444444446])

#红酒数据

In [9]:
%%time
from sklearn.datasets import load_wine
wine= load_wine()
wine_sample = pd.DataFrame(wine.data, columns=wine.feature_names)
wine_sample = pd.DataFrame(min_max_scaler.fit_transform(np.array(wine_sample)))
wine_sample['label'] = wine.target
wine_DDF=DDF(wine_sample)
wine_RDS=RDS(wine_sample)
#plot_res(wine_DDF,wine_RDS,'(b)Wine')
#plt.savefig(r'./对比实验结果图F1/wine.png')

Wall time: 2min 39s


In [10]:
wine_DDF

([0.6924339042599912,
  0.8165027259764102,
  0.8532017948952934,
  0.8515402982506582,
  0.8059411038431833],
 [0.08988742236024845,
  0.13921583850931676,
  0.24719720496894407,
  0.3726824534161491,
  0.5093206521739131])

In [11]:
wine_RDS

([0.8115946729569021,
  0.8896372781761774,
  0.9018629482896383,
  0.8388912453267292,
  0.7604999865347101],
 [0.06741847826086958,
  0.11361024844720498,
  0.21159161490683234,
  0.2827833850931677,
  0.3601708074534161])

#种子数据集

In [12]:
%%time
seeds=pd.read_csv('C:/Users/liu/python works/mydata/seeds.csv',engine='python')
seeds_sample=seeds.drop(['label'],axis=1)
seeds_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(seeds_sample)))
seeds_sample['label']=seeds.label
seeds_DDF=DDF(seeds_sample)
seeds_RDS=RDS(seeds_sample)
#plot_res(seeds_DDF,seeds_RDS,'(c)Seeds ')
#plt.savefig(r'./对比实验结果图F1/seeds.png')

Wall time: 1min 28s


In [13]:
seeds_DDF

([0.6151444054211836,
  0.7537518520885114,
  0.8286085854041403,
  0.8116228624950093,
  0.7813409476837372],
 [0.11111111111111112,
  0.1582010582010582,
  0.26560846560846557,
  0.3883597883597883,
  0.507936507936508])

In [14]:
seeds_RDS

([0.7443707419794376,
  0.8252231195228058,
  0.8368204746281535,
  0.8216156265794152,
  0.7564068221724555],
 [0.0783068783068783,
  0.12804232804232804,
  0.20582010582010582,
  0.28888888888888886,
  0.34285714285714286])

#glass

In [15]:
%%time
glass=pd.read_csv('C:/Users/liu/python works/mydata/glass.csv')
glass_sample=glass.drop(['label'],axis=1)
glass_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(glass_sample)))
glass_sample['label']=glass.label
glass_DDF=DDF(glass_sample)
glass_RDS=RDS(glass_sample)
#plot_res(glass_DDF,glass_RDS,'(d)Glass ')
#plt.savefig(r'./对比实验结果图F1/glass.png')

Wall time: 1min 59s


In [16]:
glass_DDF

([0.33631837544787957,
  0.47645148553846983,
  0.6130738431317735,
  0.6558688561132412,
  0.6652360683874182],
 [0.22093445134241718,
  0.27361928565669597,
  0.3618963154075627,
  0.46160606304926227,
  0.5683987744900427])

In [17]:
glass_RDS

([0.41355279538015266,
  0.5053479602908768,
  0.6603521512515491,
  0.6584993845133681,
  0.6811429907121548],
 [0.1504797226477465,
  0.1828388293154882,
  0.2615738127872289,
  0.31688704345722807,
  0.409566233975651])

#蛋白质数据集

In [18]:
%%time
ecoli=pd.read_csv('C:/Users/liu/python works/mydata/ecoli.csv')
ecoli_sample=ecoli.drop(['label'],axis=1)
ecoli_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(ecoli_sample)))
ecoli_sample['label']=ecoli.label
ecoli_DDF=DDF(ecoli_sample)
ecoli_RDS=RDS(ecoli_sample)
#plot_res(ecoli_DDF,ecoli_RDS,'(e)Ecoli')
#plt.savefig(r'./对比实验结果图F1/ecoli.png')

Wall time: 2min 56s


In [19]:
ecoli_DDF

([0.6888677177841265,
  0.7931807719465321,
  0.8527257874344711,
  0.8130428792661524,
  0.7880257714516581],
 [0.09436600869856139,
  0.14869187019069924,
  0.25000501840080297,
  0.3823486115757778,
  0.5061241217798594])

In [20]:
ecoli_RDS

([0.7642560393051953,
  0.827247016413402,
  0.8750979842489743,
  0.847239437219554,
  0.7471670483708708],
 [0.08006691201070593,
  0.12133154901304785,
  0.2120090331214453,
  0.2908581465373034,
  0.3590849782535965])

#图像分割

In [21]:
%%time
image=pd.read_csv('C:/Users/liu/python works/mydata/segmentation.csv')
image_sample=image.drop(['label'],axis=1)
image_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(image_sample)))
image_sample['label']=image.label
image_DDF=DDF(image_sample)
image_RDS=RDS(image_sample)
#plot_res(image_DDF,image_RDS,'(f)Image ')
#plt.savefig(r'./对比实验结果图F1/image.png')

Wall time: 4h 13min 4s


In [22]:
image_DDF

([0.6536890725636351,
  0.7859379146038192,
  0.8533788537807423,
  0.84330452386244,
  0.8296561604731124],
 [0.09980759980759982,
  0.15165945165945166,
  0.2646945646945647,
  0.40471380471380475,
  0.5565175565175565])

In [23]:
image_RDS

([0.7546804455056285,
  0.8564958047345718,
  0.8955119170292596,
  0.8865466951482514,
  0.866489136474003],
 [0.08018278018278019,
  0.1297739297739298,
  0.24011544011544012,
  0.3618566618566618,
  0.49196729196729205])

#Contracertion

In [24]:
"""
%%time
cmc=pd.read_csv('C:/Users/liu/python works/mydata/Contraception.csv',engine='python')
cmc_sample=cmc.drop(['label'],axis=1)
cmc_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(cmc_sample)))
cmc_sample['label']=cmc.label
#cmc_DDF=DDF(cmc_sample)
cmc_RDS=RDS(cmc_sample)
#plot_res(cmc_DDF,cmc_RDS,'(f)Contraception ')
#plt.savefig(r'./对比实验结果图F1/cmc.png')
"""

"\n%%time\ncmc=pd.read_csv('C:/Users/liu/python works/mydata/Contraception.csv',engine='python')\ncmc_sample=cmc.drop(['label'],axis=1)\ncmc_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(cmc_sample)))\ncmc_sample['label']=cmc.label\n#cmc_DDF=DDF(cmc_sample)\ncmc_RDS=RDS(cmc_sample)\n#plot_res(cmc_DDF,cmc_RDS,'(f)Contraception ')\n#plt.savefig(r'./对比实验结果图F1/cmc.png')\n"

#汽车数据集

In [25]:
%%time
statlog=pd.read_csv('C:/Users/liu/python works/mydata/statlog.csv',engine='python')
statlog_sample=statlog.drop(['label'],axis=1)
statlog_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(statlog_sample)))
statlog_sample['label']=statlog.label 
statlog_DDF=DDF(statlog_sample)
statlog_RDS=RDS(statlog_sample)

Wall time: 1h 11min


In [26]:
#plot_res(statlog_DDF,sRDS,'(g)Statlog')
#plt.savefig(r'./对比实验结果图F1/statlog.png')

In [27]:
statlog_RDS

([0.34520933574238183,
  0.48077978900103346,
  0.6122677598236911,
  0.6729677309310597,
  0.6855684798112924],
 [0.19884390272503713,
  0.24323362339234533,
  0.32729365629559115,
  0.39860626817180045,
  0.4687391572768253])

#新甲状腺数据

In [28]:
%%time
newthyroid=pd.read_csv('C:/Users/liu/python works/mydata/new-thyroid.csv',engine='python')
newthyroid_sample=newthyroid.drop(['label'],axis=1)
newthyroid_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(newthyroid_sample)))
newthyroid_sample['label']=newthyroid.label
newthyroid_DDF=DDF(newthyroid_sample)
newthyroid_RDS=RDS(newthyroid_sample)
#plot_res(newthyroid_DDF,newthyroid_RDS,'(h)Newthyroid')
#plt.savefig(r'./对比实验结果图F1/newthyroid.png')

Wall time: 1min 50s


In [29]:
newthyroid_DDF

([0.6269669191134708,
  0.7511821138711895,
  0.82905878462489,
  0.8241094739115674,
  0.7747841403027136],
 [0.10180011751509002,
  0.15710699214785534,
  0.2713102932535655,
  0.37157470220607874,
  0.5023262646226163])

In [30]:
newthyroid_RDS

([0.7337981298799713,
  0.843146686457804,
  0.8623824112199315,
  0.8153584596669757,
  0.7349085879776489],
 [0.08319267133165964,
  0.12143314993857166,
  0.21447305165322367,
  0.2821831098766091,
  0.3529699268201485])

#酵母菌数据

In [31]:
%%time
yeast=pd.read_csv('C:/Users/liu/python works/mydata/yeast.csv',engine='python')
yeast_sample=yeast.drop(['label'],axis=1)
yeast_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(yeast_sample)))
yeast_sample['label']=yeast.label
yeast_DDF=DDF(yeast_sample)
yeast_RDS=RDS(yeast_sample)
#plot_res(yeast_DDF,yeast_RDS,'(i)Yeast')
#plt.savefig(r'./对比实验结果图F1/yeast.png')

Wall time: 1h 12min 21s


In [32]:
yeast_DDF

([0.17214671665466938,
  0.2818151574061698,
  0.42731925923082315,
  0.521152399063553,
  0.5881566554630564],
 [0.41138042249398854,
  0.44101669430878443,
  0.5114447932388717,
  0.5599563273668444,
  0.623528607200724])

In [33]:
yeast_RDS

([0.2051067511385125,
  0.33516905229025185,
  0.45191547364725704,
  0.5066642461873022,
  0.5320174368291268],
 [0.2679960708020806,
  0.29313241626542563,
  0.3237430488203798,
  0.36864245637997295,
  0.4090427027678464])

#vowel

In [34]:
%%time
vowel=pd.read_csv('C:/Users/liu/python works/mydata/vowel.csv',engine='python')
vowel_sample=vowel.drop(['label'],axis=1)
vowel_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(vowel_sample)))
vowel_sample['label']=vowel.label
vowel_DDF=DDF(vowel_sample)
vowel_RDS=RDS(vowel_sample)
#plot_res(vowel_DDF,vowel_RDS,'(i)vowel')
#plt.savefig(r'./对比实验结果图F1/vowel.png')

Wall time: 51min 24s


#letter数据集

In [35]:
%%time
letter=pd.read_csv('C:/Users/liu/python works/mydata/letter.csv',engine='python')
letter_sample=letter.drop(['label'],axis=1)
letter_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(letter_sample)))
letter_sample['label']=letter.label
letter_DDF=DDF(letter_sample)
letter_RDS=RDS(letter_sample)
#plot_res(letter_DDF,letter_RDS,'(j)Letter')
#plt.savefig(r'./对比实验结果图F1/letter.png')

Wall time: 4h 9min 25s


#pen_base

In [36]:
%%time
pen_base=pd.read_csv('C:/Users/liu/python works/mydata/pen_base.csv',engine='python')
pen_sample=pen_base.drop(['label'],axis=1)
pen_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(pen_sample)))
pen_sample['label']=pen_base.label
pen_DDF=DDF(pen_sample)
pen_RDS=RDS(pen_sample)
#plot_res(pen_DDF,pen_RDS,'(j)Pen')
#plt.savefig(r'./对比实验结果图F1/pen.png')

Wall time: 8h 24min 8s


In [37]:
Fh_DDF=np.array(iris_DDF[0])+np.array(wine_DDF[0])+np.array(seeds_DDF[0])+np.array(glass_DDF[0])+np.array(ecoli_DDF[0])+np.array(image_DDF[0])+np.array(statlog_DDF[0])+np.array(yeast_DDF[0])+np.array(letter_DDF[0])+np.array(newthyroid_DDF[0])+np.array(pen_DDF[0])+np.array(vowel_DDF[0])
Re_DDF=np.array(iris_DDF[1])+np.array(wine_DDF[1])+np.array(seeds_DDF[1])+np.array(glass_DDF[1])+np.array(ecoli_DDF[1])+np.array(image_DDF[1])+np.array(statlog_DDF[1])+np.array(yeast_DDF[1])+np.array(letter_DDF[1])+np.array(newthyroid_DDF[1])+np.array(pen_DDF[1])+np.array(vowel_DDF[1])

In [38]:
Fh_RDS=np.array(iris_RDS[0])+np.array(wine_RDS[0])+np.array(seeds_RDS[0])+np.array(glass_RDS[0])+np.array(ecoli_RDS[0])+np.array(image_RDS[0])+np.array(statlog_RDS[0])+np.array(yeast_RDS[0])+np.array(letter_RDS[0])+np.array(newthyroid_RDS[0])+np.array(pen_RDS[0])+np.array(vowel_RDS[0])
Re_RDS=np.array(iris_RDS[1])+np.array(wine_RDS[1])+np.array(seeds_RDS[1])+np.array(glass_RDS[1])+np.array(ecoli_RDS[1])+np.array(image_RDS[1])+np.array(statlog_RDS[1])+np.array(yeast_RDS[1])+np.array(letter_RDS[1])+np.array(newthyroid_RDS[1])+np.array(pen_RDS[1])+np.array(vowel_RDS[1])

In [39]:
#Re_DDF=np.array([0.9,1.4,2.4,3.1,3.6])

In [40]:
#Re_RDS

In [41]:
import mpl_toolkits.axisartist.axislines as axislines
import matplotlib as mpl
mpl.rcParams['font.sans-serif'] = [u'SimHei']
mpl.rcParams['axes.unicode_minus'] = False
%matplotlib
noise_rate=[5,10,20,30,40]
fig = plt.figure(figsize=(6,8))
#ax1 = axislines.Subplot(fig, 2,1,1)
#fig.add_subplot(ax1)
#ax1.set_xticks(noise_rate)
#ax1.axis([0,40,0,1])
plt.subplot(2,1,1)
plt.plot(noise_rate,Fh_DDF/12,marker='o',color='green',markersize=4,label='DDF')
plt.plot(noise_rate,Fh_RDS/12,marker='*',color='red',markersize=5,label='DDF-NOS')
plt.xticks(noise_rate,['5','10','20','30','40'])
plt.yticks([0,0.2,0.4,0.6,0.8,1.0],['0','0.2','0.4','0.6','0.8','1.0'])
plt.xlabel('noise rate/% \n(a)',fontsize=12)
plt.ylabel('F1',fontsize=12)
plt.legend(frameon=False)
plt.show()
#ax2 = axislines.Subplot(fig, 2,1,2)
plt.subplot(2,1,2)
plt.plot(noise_rate,Re_DDF/12,marker='o',color='green',markersize=4,label='DDF')
plt.plot(noise_rate,Re_RDS/12,marker='*',color='red',markersize=5,label='DDF-NOS')
plt.xticks(noise_rate,['5','10','20','30','40'])
plt.yticks([0,0.2,0.4,0.6,0.8,1.0],['0','0.2','0.4','0.6','0.8','1.0'])
plt.xlabel('noise rate/%\n(b)',fontsize=12)
plt.ylabel('Re',fontsize=12)
# plt.title('不同噪声下的过滤值Re',fontsize=12)
plt.legend(frameon=False)
plt.show()
plt.tight_layout(2) #设置子图间隔
#plt.suptitle(title)
plt.show()
plt.savefig(r'./对比实验结果图F1/真实.png')

Using matplotlib backend: Qt5Agg
